## Preliminary

This notebook explains you how to build your 1st plugin. The plugin is a Python script uploaded which will be triggered based on an event you define. For instance you can trigger a plugin each time a labeler click on submit.
The plugin should have separate methods for the different types of events : at the moment these are *onSubmit* and *onReview*. These methods will have a predefined set of parameters

The final plugin will have a global instance of *kili* that you can use directly (it will be instantiated with a separate *API_KEY* belonging to the plugin)

Therefore, the skeleton of the plugin should be :

```python
def onSubmit(label, projectId, assetId):
    """
    Function called when a DEFAULT label is submitted by a labeler

    Parameters
    ----------
    label : dict
        A dictionary that has the most important fields of a Label: 
            [id, jsonResponse, authorId, labelType, createdAt, secondsToLabel] 
            and some technical fields
    projectId : string
        The id of the project that the plugin runs on.
    assetId : string
        The id of the asset that the plugin runs on.
    """
    pass

def onReview(label, projectId, assetId):
    """
    Function called when a REVIEW label is submitted by a reviewer

    Parameters
    ----------
    label : dict
        A dictionary that has the most important fields of a Label: 
            [id, jsonResponse, authorId, labelType, createdAt, secondsToLabel] 
            and some technical fields
    projectId : string
        The id of the project that the plugin runs on.
    assetId : string
        The id of the asset that the plugin runs on.
    """
    pass
```

**IMPORTANT** : Imports of packages inside the plugin are not permitted at the moment (except for `numpy`).

## Instantiate Kili with your personal API_KEY

In [ ]:
#%pip install kili

from kili.client import Kili
import os

api_endpoint = os.getenv('KILI_API_ENDPOINT')
# If you use Kili SaaS, use the url 'https://cloud.kili-technology.com/api/label/v2/graphql'
# api_endpoint = 'https://cloud.kili-technology.com/api/label/v2/graphql'

api_key = input()

kili = Kili(api_endpoint=api_endpoint, api_key=api_key)

## Develop your future plugin

The first step is to define the functions that will be called when the event is triggered. You will be able to iterate on these functions locally with the help of the next section. 

This cell should be the content of the `.py` file that you will upload as a plugin at the end.

In [ ]:
def onSubmit(label, projectId, assetId):
    """
    Function called when a DEFAULT label is submitted by a labeler

    Parameters
    ----------
    label : dict
        A dictionary that has the most important fields of a Label: 
            [id, jsonResponse, authorId, labelType, createdAt, secondsToLabel] 
            and some technical fields
    projectId : string
        The id of the project that the plugin runs on.
    assetId : string
        The id of the asset that the plugin runs on.
    """
    #print(label, projectId, assetId)
    pass

def onReview(label, projectId, assetId):
    """
    Function called when a REVIEW label is submitted by a reviewer

    Parameters
    ----------
    label : dict
        A dictionary that has the most important fields of a Label: 
            [id, jsonResponse, authorId, labelType, createdAt, secondsToLabel] 
            and some technical fields
    projectId : string
        The id of the project that the plugin runs on.
    assetId : string
        The id of the asset that the plugin runs on.
    """
    #print(label, projectId, assetId)
    #print(label["unexisting_field"]) # will throw error - to see an example how error messages will be in the logs
    pass

### Testing locally the plugin

This section will allow you to test your plugin locally before uploading it.

In [ ]:
import traceback

def runPlugin(label, project_id, asset_id, call_on_submit = True, call_on_review = True):
    """
    This function is a wrapper around the actual onSubmit and onReview functions.
    In case there is an error, it will catch it and print its name. The trace of the
    error will not be saved in the actual logs of the plugin that you will be able to
    retrieve after uploading the plugin, but here they will be printed to help you
    in the development of the plugin.

    If you want to have the full trace of the error, you can call directly the functions
    onSubmit and onReview without using this wrapper, for example :
    >>> onSubmit(projectId=project_id,assetId=asset_id,label=label)

    You can also pass the values of call_on_submit and call_on_review to test
    only certain events.
    """
    try:
        if call_on_submit:
            onSubmit(projectId=project_id,assetId=asset_id,label=label)

        if call_on_review:
            onReview(projectId=project_id,assetId=asset_id,label=label)
    except Exception as e:
        print(repr(e))
        traceback.print_exc() # This will not be saved in the logs of the actual plugin,
                              # but it can help when iterating over the development of the plugin

def get_label(label_id, project_id):
    """
    Function to get the object Label with the same keys as it will be in the plugin
    """
    label = kili.labels(
            project_id=project_id,
            label_id=label_id,
            fields=['id', 'jsonResponse', 'author.id', 'labelType', 'createdAt', 'secondsToLabel']
        )[0]

    label['authorId'] = label['author']['id']
    del label['author']
    return label

### Test the plugin run

If you already have created a project for testing and there are some labels created, you can use directly their ids and use the following cell. Otherwise, you can follow the notebook *plugins_example.ipynb* to create a new project, upload an asset and an associated label.

In [ ]:
project_id = ''
asset_id = ''
label_id = ''

label = get_label(label_id=label_id, project_id=project_id)

runPlugin(project_id=project_id, asset_id=asset_id,label=label)

## Uploading the plugin
Once you finished writing and testing your functions, you can upload this plugin to Kili and activate it on a project (you can re-use the same plugin on different projects if needed).

First, you have to create a python file (for example `plugin.py`), and copy inside only the functions of the plugin (the content of the cell *Develop your future plugin*). You do not need any other info related to the *Kili* instance since it wil be present globally.

Then, you can use the python sdk or the CLI to import this plugin :

In [ ]:
path_to_plugin = 'path/to/file/plugin.py'
plugin_name = 'plugin name'
project_id = 'project_id'

kili.upload_plugin_beta(path_to_plugin, plugin_name)

kili.activate_plugin_on_project(plugin_name, project_id=project_id)

## Monitoring the plugin
Important to know is that the creation of the plugin takes some time (around 5 minutes), so after that time the plugin will begin to be run (if labeling events will be triggered on this project) and you will be able to get the logs of the runs. To do that, you can use:

In [ ]:
import json
from datetime import date
from datetime import datetime
dt = date.today()  # You can change this date if needed
start_date = datetime.combine(dt, datetime.min.time())

logs = kili.get_plugin_logs(project_id=project_id, plugin_name=plugin_name, start_date=start_date)

logs_json = json.loads(logs)
print(json.dumps(logs_json, indent=4))

## Managing your plugin
You also have several other methods to manage your plugins and their lifecycle.

In [ ]:
# Get the list of all uploaded plugins in your organization
plugins = kili.list_plugins()

In [ ]:
# Get the status of a plugin
status = kili.get_plugin_status(plugin_name=plugin_name)

In [ ]:
# Update a plugin with new source code
updated_path = 'path/to/updated/file.py'
kili.update_plugin(plugin_name=plugin_name, file_path=updated_path)

In [ ]:
# Deactivate the plugin on a certain project (the plugin can still be active for other projects)
kili.deactivate_plugin_on_project(plugin_name=plugin_name, project_id=project_id)

In [ ]:
# Delete the plugin completely (deactivates automatically the plugin from all projects)
kili.delete_plugin(plugin_name=plugin_name)